Imports

In [15]:
import os
import random
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go

from typing import List, Optional
from dataclasses import dataclass

Constants

In [16]:
PMC_CONVENTIONAL = 4362.47
PMC_OPERATIVE_69 = 2024.481
PMC_OPERATIVE_138 = 2513.486

PQ_QUANTITY = 72

Classes and functions

In [17]:
from dataclasses import dataclass
import pandas as pd

@dataclass
class BarData:
    '''
    A dataclass that retains relevant information about system's bars.

    This dataclass stores information about system bars, including the file name, name, voltage class, and a DataFrame.

    Attributes:
        _file_name (str): The file name associated with the bar data.

    Properties:
        file_name (str): The file name associated with the bar data.
        name (str): The name extracted from the file name (excluding extension).
        voltage_class (str): The voltage class extracted from the file name.
        df (pd.DataFrame): A DataFrame containing the bar data.
    '''

    _file_name: str

    @property
    def file_name(self) -> str:
        '''str: The file name associated with the bar data.'''
        return self._file_name

    @property
    def name(self) -> str:
        '''str: The name extracted from the file name (excluding extension).'''
        return self._file_name[9:-4]

    @property
    def voltage_class(self) -> str:
        '''str: The voltage class extracted from the file name.'''
        return self.name[-3:]

    @property
    def df(self) -> pd.DataFrame:
        '''pd.DataFrame: A DataFrame containing the bar data.'''
        return self.read_csv(self._file_name)
    
    @property
    def power_threshold(self) -> float:
        '''
        Get the power threshold for operational voltage based on a filtered DataFrame.

        Returns:
            float: Power threshold for operational voltage.
        '''
        filtered_df = self.df.loc[self.df[1].between(0.95, 0.96)] 
        
        if not filtered_df.empty:
            voltage_threshold_index = filtered_df[1].idxmin()
            power_threshold = filtered_df.at[voltage_threshold_index, 0]
        else:
            power_threshold = PMC_CONVENTIONAL

        return power_threshold
    
    def read_csv(self, _file_name: str) -> pd.DataFrame:
        '''
        Read bar data from a CSV file and return it as a DataFrame.

        Args:
            _file_name (str): The file name of the CSV file.

        Returns:
            pd.DataFrame: A DataFrame containing the bar data.
        '''
        df = pd.read_csv(_file_name, delimiter= ';', header= None)
        return df
    
    def calculate_operational_vsm(self, operational_point_power: float) -> float:
        '''
        Calculate the Voltage Stability Margin (VSM) based on the operational point power.

        Parameters:
            operational_point_power (float): The operational point power.

        Returns:
            float: The calculated Voltage Stability Margin (VSM).
        '''
        return 100 * ((self.power_threshold - operational_point_power) / self.power_threshold)

    def __repr__(self) -> str:
        '''str: A string representation of the BarData instance.'''
        return f'{self.name}/Voltage class: {self.voltage_class}/PTOV: {self.power_threshold}'

    
def get_random_sum(loads: List[int], get_amount: int) -> int:
    '''
    Calculate the sum of a random selection of load values in megawatts (MW).

    This function randomly selects a specified number of load values from the provided list and calculates their sum in megawatts (MW).

    Args:
        loads (List[int]): A list of load values in megawatts (MW).
        get_amount (int): The number of load values to randomly select and sum.

    Returns:
        int: The sum of the randomly selected load values in megawatts (MW).
    '''
    return sum(random.sample(loads, get_amount))

def increment_power(load: float,
                    increment_amount: int,
                    increment_percentage: float
                    ) -> List[float]:
    '''
    Increment a load value by a specified number and percentage multiple times.

    This function takes a starting load value and increments it by a specified number and percentage for a given number of times.

    Args:
        load (float): The initial load value.
        increment_amount (int): The number of times to increment the load.
        increment_percentage (float): The percentage by which to increment the load in each step.

    Returns:
        list: A list containing the incremented load values, including the initial load.
    '''
    incremented_values = [load]

    for _ in range(increment_amount):
        load *= (1 + increment_percentage)
        incremented_values.append(load)

    return incremented_values

def make_matrices(max_power: float,
                  load_value: List[int],
                  charging_station_power: List[float],
                  penetration_level: int,
                  increment: bool = False
                  ) -> tuple[List[List], List[List]]:
    '''
    Generate matrices of power and margin of stability based on max power, load, charging station power, and increment settings.

    This function calculates power and margin of stability matrices for a given load value and power value reference, considering various charging station power scenarios.

    Args
        max_power (float): Reference power value.
        load_value (float): The base load value.
        charging_station_power (list): A list of charging station power values.
        increment (bool, optional): Whether to increment load with charging station power scenarios (default is False).

    Returns:
        tuple: A tuple containing two lists of lists:
            - A power matrix: List of lists representing power values for different scenarios.
            - A margin of stability matrix: List of lists representing margin of stability values for different scenarios.
    '''
    pow_matrix = []
    vsm_matrix = []
    inserted_power_matrix = []

    for _ in range(PQ_QUANTITY):
        pow_row = []
        vsm_row = []
        inserted_power_row = []

        for _ in range(200):
            charging_site_power = get_random_sum(charging_station_power, get_amount= 3) / 1000
            
            power_value = load_value + (insertion_power := (penetration_level * (_ + 1) * charging_site_power if increment else load_value))

            vsm_value = 100.0 * ((max_power - power_value) / max_power)

            pow_row.append(power_value)
            vsm_row.append(vsm_value)
            inserted_power_row.append(insertion_power)

        pow_matrix.append(pow_row)
        vsm_matrix.append(vsm_row)
        inserted_power_matrix.append(inserted_power_row)
    
    return pow_matrix, vsm_matrix, inserted_power_matrix

def filter_by_voltage(bars: List[BarData],
                      voltage_class: Optional[str] = None
                      ) -> List[BarData]:
    '''
    Filter a list of BarData instances by their voltage_class attribute.

    Args:
        bars (List[BarData]): List of BarData instances to filter.
        voltage_class (str): The voltage class to filter by. Defaults to None.

    Returns:
        List[BarData]: List of filtered BarData instances.
    '''

    if voltage_class is None:
        return bars
    
    filtered_bars = [bar for bar in bars if bar.voltage_class == voltage_class]
    
    return filtered_bars

def matrix_to_list(target_matrix: List[List[float]]) -> List[float]:
    '''
    Turns a matrix into a list.

    Args:
        target_matrix (List[List[float]]): Matrix to be turned into a list.

    Returns:
        List[float]: List of elements of target_matrix.
    '''

    matrix_as_list = [element for row in target_matrix for element in row]

    return matrix_as_list

def save_as_csv(target_data, file_name, folder_path):
    file_path = os.path.join(folder_path, file_name)

    pd.DataFrame(target_data).to_csv(file_path, index= False, header= None)

BarData objects

In [18]:
folder_path = r'PVs_csvs'

bars = []

for file in os.listdir(folder_path):
    if file.endswith('.csv'):
        file_path = os.path.join(folder_path, file)
        
        bar = BarData(file_path)

        bars.append(bar)

Load defintions

In [19]:
heavy_load = 1642.69
charging_station_power = [114, 147, 180, 213]

incremented_loads = increment_power(heavy_load, 5, 0.082)

incremented_loads

[1642.69,
 1777.3905800000002,
 1923.1366075600004,
 2080.8338093799207,
 2251.4621817490743,
 2436.0820806524985]

Yearly analyses

Conventional VSM

In [27]:
years = [2026, 2028, 2030, 2032, 2034, 2036]
penetration_levels = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

pow_matrix = {}
vsm_matrix = {}

insertion_matrix = {} 

pow_matrix_unincremented = {}
vsm_matrix_unincremented = {}

insertion_matrix_unincremented = {}

folder_path = r'C:\Users\luizsousa\Documents\TCC\6Case met analyses\VSMs_csvs'

for year in years:

    load_data = incremented_loads[years.index(year)]

    pow_matrix[year] = {}
    vsm_matrix[year] = {}

    insertion_matrix[year] = {} 

    pow_matrix_unincremented[year] = {}
    vsm_matrix_unincremented[year] = {}
    insertion_matrix_unincremented[year] = {}
    
    for penetration_level in penetration_levels:
        pow_matrix[year][penetration_level], \
        vsm_matrix[year][penetration_level], \
        insertion_matrix[year][penetration_level] = make_matrices(max_power= PMC_CONVENTIONAL,
                                                                  load_value= load_data,
                                                                  charging_station_power= charging_station_power,
                                                                  penetration_level= penetration_level,
                                                                  increment= True
                                                                  )
    
        file_name = f'vsm_matrix_{year}_pen_level_{penetration_level}.csv'
        file_path = os.path.join(folder_path, file_name)

        pd.DataFrame(matrix_to_list(vsm_matrix[year][penetration_level])) \
            .to_csv(file_path,
                    index= False,
                    header= None)
        
        pow_matrix_unincremented[year], \
        vsm_matrix_unincremented[year], \
        insertion_matrix_unincremented[year] = make_matrices(max_power = PMC_CONVENTIONAL,
                                                             load_value= load_data,
                                                             charging_station_power= charging_station_power,
                                                             penetration_level= 1,
                                                             increment= False
                                                             )


Operative 69kV reference VSM

In [29]:
years = [2026, 2028, 2030, 2032, 2034, 2036]

pow_matrix_incremented_69 = {}
vsm_matrix_incremented_69 = {}

insertion_matrix_incremented_69 = {}

pow_matrix_unincremented_69 = {}
vsm_matrix_unincremented_69 = {}

insertion_matrix_unincremented_69 = {}

for year in years:
    load_data = incremented_loads[years.index(year)]
    
    pow_matrix_incremented_69[year], \
    vsm_matrix_incremented_69[year], \
    insertion_matrix_incremented_69[year] = make_matrices(max_power= PMC_OPERATIVE_69,
                                                    load_value= load_data,
                                                    charging_station_power= charging_station_power,
                                                    penetration_level= 1,
                                                    increment= True)
    
    pow_matrix_unincremented_69[year], \
    vsm_matrix_unincremented_69[year], \
    insertion_matrix_incremented_69[year] = make_matrices(max_power= PMC_OPERATIVE_69,
                                                          load_value= load_data,
                                                          charging_station_power= charging_station_power,
                                                          penetration_level= 1,
                                                          increment= False)


Operative 138kV reference VSM

In [ ]:
years = [2026, 2028, 2030, 2032, 2034, 2036]

pow_matrix_incremented_138 = {}
vsm_matrix_incremented_138 = {}

insertion_matrix_incremented_138 = {}

pow_matrix_unincremented_138 = {}
vsm_matrix_unincremented_138 = {}

insertion_matrix_unincremented_138 = {}
for year in years:
    load_data = incremented_loads[years.index(year)]
    
    pow_matrix_incremented_138[year], \
    vsm_matrix_incremented_138[year], \
    insertion_matrix_incremented_138[year] = make_matrices(max_power= PMC_OPERATIVE_138,
                                                    load_value= load_data,
                                                    charging_station_power= charging_station_power,
                                                    penetration_level= 1,
                                                    increment= True)
    
    pow_matrix_unincremented_138[year], \
    vsm_matrix_unincremented_138[year], \
    insertion_matrix_incremented_138[year] = make_matrices(max_power= PMC_OPERATIVE_138,
                                                          load_value= load_data,
                                                          charging_station_power= charging_station_power,
                                                          penetration_level= 1,
                                                          increment= False)


Plots

In [ ]:
fig = go.Figure()

voltage_to_be_filtered = None

filtered_bars = filter_by_voltage(bars, voltage_to_be_filtered)

for index, bar in enumerate(filtered_bars):
    data = bar.df.T
    
    fig.add_trace(
        go.Scatter(x= data.values[0],
                   y= data.values[1],
                   mode= 'lines',
                   name= f'Bar {index + 1}: {bar.name}')
    )

fig.update_layout(
    xaxis_title= 'Load (MW)',
    yaxis_title= 'Voltage (pu)',
    title= f'PxV curves for all voltage levels',
    title_x= 0.5
)

fig.show()


In [ ]:
fig = go.Figure()

voltage_to_be_filtered = '230'

filtered_bars = filter_by_voltage(bars, voltage_to_be_filtered)

for index, bar in enumerate(filtered_bars):
    data = bar.df.T
    
    fig.add_trace(
        go.Scatter(x= data.values[0],
                   y= data.values[1],
                   mode= 'lines',
                   name= f'Bar {index + 1}: {bar.name}')
    )

fig.update_layout(
    xaxis_title= 'Load (MW)',
    yaxis_title= 'Voltage (pu)',
    title= f'PxV curves for 230kv voltage level',
    title_x= 0.5
)

fig.show()


In [ ]:
fig = go.Figure()

voltage_to_be_filtered = '138'

filtered_bars = filter_by_voltage(bars, voltage_to_be_filtered)

for index, bar in enumerate(filtered_bars):
    data = bar.df.T
    
    fig.add_trace(
        go.Scatter(x= data.values[0],
                   y= data.values[1],
                   mode= 'lines',
                   name= f'Bar {index + 1}: {bar.name}')
    )

fig.update_layout(
    xaxis_title= 'Load (MW)',
    yaxis_title= 'Voltage (pu)',
    title= f'PxV curves for 138kV voltage level',
    title_x= 0.5
)

fig.show()


In [ ]:
fig = go.Figure()

voltage_to_be_filtered = '069'

filtered_bars = filter_by_voltage(bars, voltage_to_be_filtered)

for index, bar in enumerate(filtered_bars):
    data = bar.df.T
    
    fig.add_trace(
        go.Scatter(x= data.values[0],
                   y= data.values[1],
                   mode= 'lines',
                   name= f'Bar {index + 1}: {bar.name}')
    )

fig.update_layout(
    xaxis_title= 'Load (MW)',
    yaxis_title= 'Voltage (pu)',
    title= f'PxV curves for 69kV voltage level',
    title_x= 0.5
)

fig.show()


In [ ]:
fig = go.Figure()

vsm_per_year = []
vsm_per_year_69 = []
vsm_per_year_138 = []

for year in years:
    vsm_per_year.append(vsm_matrix_unincremented[year][0][0])
    vsm_per_year_69.append(vsm_matrix_unincremented_69[year][0][0])
    vsm_per_year_138.append(vsm_matrix_unincremented_138[year][0][0])

fig.add_trace(
    go.Scatter(x= years,
               y= vsm_per_year,
               name= 'Conventional VSM')
)

fig.add_trace(
    go.Scatter(x= years,
               y= vsm_per_year_69,
               name= '69kV operative VSM')
)

fig.add_trace(
    go.Scatter(x= years,
               y= vsm_per_year_138,
               name= '138kV operative VSM')
)

fig.update_layout(
    xaxis_title= 'Years',
    yaxis_title= 'Voltage margin (%)',
    title= 'Voltage stability margin over the years (unincremented load)',
    title_x= 0.5
)

fig.show()

In [ ]:
fig = go.Figure()


for year in years:
    for penetration_level in penetration_levels:
        fig.add_trace(
            go.Box(y= matrix_to_list(vsm_matrix[year][penetration_level]),
            name= f'{year}:pen_level: {penetration_level}'
            )
        )

fig.update_layout(
    title= 'Voltage stability margin over the years (incremented load)',
    title_x= 0.5,
    xaxis= dict(title= 'Years'),
    yaxis=dict(
        title='Voltage margin (%)',
        tickmode='linear',  # Use a linear tick mode
        tick0=0,            # Starting tick value
        dtick=5
        )
)

fig.show()

In [ ]:
x_data = years
y_data = [[number for number in range(len(bars))] for _ in range(len(years))]
z_data = [[bar.calculate_operational_vsm(incremented_loads[_]) for bar in bars] for _ in range(len(years))]

fig = plt.figure(figsize=(20,20))
ax = fig.add_subplot(111, projection='3d')

for x, y, z in zip(x_data, y_data, z_data):
    ax.bar3d(x, y, 0, dx=0.5, dy=0.5, dz=z, shade=True)

ax.set_xlabel('X-axis')
ax.set_ylabel('Y-axis')
ax.set_zlabel('Z-axis')

ax.set_yticks(range(len(bars)))
ax.set_yticklabels([f'{bar.name}' for bar in bars])

ax.tick_params(axis='y', labelrotation=90)

ax.set_title('Operational VSM for all bars')

plt.show()

save_as_csv(z_data, 'all_bars_vsm.csv', r'C:\Users\luizsousa\Documents\TCC\6Case met analyses\VSMs_by_voltage_csvs')

In [ ]:
filtered_bars = filter_by_voltage(bars, '069')

x_data = years
y_data = [[number for number in range(len(filtered_bars))] \
          for _ in range(len(years))]
z_data = [[bar.calculate_operational_vsm(incremented_loads[_]) for bar in filtered_bars] \
          for _ in range(len(years))]

fig = plt.figure(figsize=(20,20))
ax = fig.add_subplot(121, projection='3d')

for x, y, z in zip(x_data, y_data, z_data):
    ax.bar3d(x, y, 0, dx=0.5, dy=0.5, dz=z, shade=True)

ax.set_xlabel('X-axis')
ax.set_ylabel('Y-axis')
ax.set_zlabel('Z-axis')

ax.set_yticks(range(len(filtered_bars)))
ax.set_yticklabels([f'{bar.name}' for bar in filtered_bars])

ax.tick_params(axis='y', labelrotation=90)

ax.set_title('Operational VSM for 69kV bars')

plt.show()

save_as_csv(z_data, '69_bars_vsm.csv', r'C:\Users\luizsousa\Documents\TCC\6Case met analyses\VSMs_by_voltage_csvs')


In [ ]:
filtered_bars = filter_by_voltage(bars, '138')

x_data = years
y_data = [[number for number in range(len(filtered_bars))] \
          for _ in range(len(years))]
z_data = [[bar.calculate_operational_vsm(incremented_loads[_]) for bar in filtered_bars] \
          for _ in range(len(years))]

fig = plt.figure(figsize=(20,20))
ax = fig.add_subplot(111, projection='3d')

for x, y, z in zip(x_data, y_data, z_data):
    ax.bar3d(x, y, 0, dx=0.5, dy=0.5, dz=z, shade=True)

ax.set_xlabel('Years')
ax.set_ylabel('Bar name')
ax.set_zlabel('Operational')

ax.set_yticks(range(len(filtered_bars)))
ax.set_yticklabels([f'{bar.name}' for bar in filtered_bars])

ax.tick_params(axis='y', labelrotation=90)

ax.set_title('Operational VSM for 138kV bars')

plt.show()

save_as_csv(z_data, '138_bars_vsm.csv', r'C:\Users\luizsousa\Documents\TCC\6Case met analyses\VSMs_by_voltage_csvs')
